In [2]:
import cv2
import numpy as np
import time 

subtractor = cv2.createBackgroundSubtractorMOG2(history=10, varThreshold=500, detectShadows=False)

def canny(img):
    if img is None:
        cap.release()
        cv2.destroyAllWindows()
        exit()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = 5
    blur = cv2.GaussianBlur(gray,(kernel, kernel),1)
    canny = cv2.Canny(gray, 50, 10)
    return canny

def region_of_interest(canny):
    height = canny.shape[0]
    width = canny.shape[1]
    mask = np.zeros_like(canny)
    triangle = np.array([[
    (200, height),
    (800, 350),
    (1200, height),]], np.int32)
    cv2.fillPoly(mask, triangle, 100)
    masked_image = cv2.bitwise_and(canny, mask)
    return masked_image

def houghLines(cropped_canny):
    return cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 100, 
        np.array([]), minLineLength=40, maxLineGap=5)

def addWeighted(frame, line_image):
    return cv2.addWeighted(frame, 0.8, line_image, 1, 1)
 
def display_lines(img,lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),20)
                cv2.line(img,(x2,y2),(x1,y1),(0,0,0),10)
                cv2.circle(img,(x1,y1), 20, (0,0,0), -1)
                cv2.circle(img,(x2,y2), 20,  (0,0,0), -1)
                cv2.circle(img,(x1,y2), 20,  (0,0,0), -1)
                pts = np.array([[x1,x2],[x2,y2],[x1,y1],[x1,y2]], np.int32)
                pts = pts.reshape((-1,1,2))
                cv2.polylines(img,[pts],True,(0,0,0),10)
                #cv2.fillPoly(img, pts, (0,0,0)[[ shift[ offset]]])
                a3 = np.array( [[[x1,y1],[x2,y2],[x2,y2],[x2+x1/2,y2+y1/2]]], dtype=np.int32 )
                a31 = np.array( [[[x1,y1],[x2,y2],[y1,y1],[x1,x2]]], dtype=np.int32 )
                cv2.fillPoly(img, a3, (255,242,253))
                cv2.fillPoly(img, a31, (255,242,253))
                cv2.putText(img,'Starting lane ',(x1,y1),cv2.FONT_HERSHEY_PLAIN,2,(0,0,0),2,cv2.LINE_AA)
                cv2.putText(img,'End lane ',(x2,y2),cv2.FONT_HERSHEY_PLAIN,2,(0,0,0),2,cv2.LINE_AA)
    return line_image
 
def make_points(image, line):
    slope, intercept = line
    y1 = int(image.shape[0])
    y2 = int(y1*3.0/5)      
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return [[x1, y1, x2, y2]]
 
def average_slope_intercept(image, lines):
    left_fit    = []
    right_fit   = []
    if lines is None:
        return None
    for line in lines:
        for x1, y1, x2, y2 in line:
            fit = np.polyfit((x1,x2), (y1,y2), 1)
            slope = fit[0]
            intercept = fit[1]
            if slope < 0: 
                left_fit.append((slope, intercept))
            else:
                right_fit.append((slope, intercept))
    left_fit_average  = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    left_line  = make_points(image, left_fit_average)
    right_line = make_points(image, right_fit_average)
    averaged_lines = [left_line, right_line]
    return averaged_lines
car_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+"cas2.xml")
cap = cv2.VideoCapture("test1.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    
    mask = subtractor.apply(frame)
    
    cv2.imshow("mask", mask)
    first_frame=0
    next_frame=0
    next_frame=time.time()
    fps=float(1/(next_frame-first_frame))
    first_frame=next_frame
        
    canny_image = canny(frame)
    cropped_canny = region_of_interest(canny_image)
    cv2.imshow("cropped_canny",cropped_canny)
    cv2.imshow("cropped",canny_image)
    
    lines = houghLines(cropped_canny)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = addWeighted(frame, line_image)
    cv2.imshow("result1", line_image)
    window_name = 'Image'

    # font 
    font = cv2.FONT_HERSHEY_SIMPLEX 

    # org 
    org = (600,550) 

    # fontScale 
    fontScale = 1
    fontScale1 = 0.6
    # Blue color in BGR 
    color = (0,0,0) 



    # Line thickness of 2 px 
    thickness = 2
      # Detects cars of different sizes in the input image 
    '''cars = car_cascade.detectMultiScale(gray)
    for (x,y,w,h) in cars:
        cv2.rectangle(img,(x,y),(x+w+5,y+h+5),(0,0,255),3)
    cv2.imshow('Car Detector', combo_image)
    '''
    combo_image= cv2.putText(combo_image,'***SMART LANE DETECTOR*** ',(330,50),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),2,cv2.LINE_AA)
    combo_image= cv2.putText(combo_image,'[PRESS A tO EXIT] ',(370,100),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),0,cv2.LINE_AA)

    combo_image= cv2.putText(combo_image,'FPS',(5,100),cv2.FONT_HERSHEY_PLAIN,2,(255,120,60),2,cv2.LINE_AA)
    combo_image= cv2.putText(combo_image,str(fps),(10,150),cv2.FONT_HERSHEY_PLAIN,2,(255,120,60),2,cv2.LINE_AA)
    #combo_image = cv2.putText(combo_image, '[Far Field]', (700,396),font,fontScale, color, thickness, cv2.LINE_AA) 
    combo_image = cv2.putText(combo_image, '[Near Field]', (700,656),font,fontScale, color, thickness, cv2.LINE_AA) 
    combo_image = cv2.putText(combo_image, '<--LANE[region of intersest]-->', org,font,fontScale1, color, thickness, cv2.LINE_AA) 
    cv2.imshow("result", combo_image)
    #cv2.imshow("region for smart car",combo_image[400:1000])

    if cv2.waitKey(1) & 0xFF == ord('a'):
        print('end')
        break

cap.release()
cv2.destroyAllWindows()

end


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 135)

In [46]:
print(func((-2,-1,1,2,3)))

5


In [30]:
k=[print(i) for i in my_string: if i not in "aieou"]

SyntaxError: invalid syntax (<ipython-input-30-198a56bc07bb>, line 1)

In [31]:
a=564
b='456'

In [32]:
print(a+b)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [33]:
import pandas  as pd